In [1]:
import gzip
import io
import numpy as np
from astropy.io import fits

In [2]:
testfile = "test.fits"
testfile_gz = "test.fits.gz"

In [3]:
hdul = fits.HDUList(
    [fits.PrimaryHDU()]
    + [
        fits.ImageHDU(
            data=np.random.randn(1000, 1000).astype(np.float32),
            header=fits.Header({f"KEY{i}{k}": k for k in range(1000)}),
            name=f"IMAGE{i}",
        )
        for i in range(20)
    ]
)
hdul.writeto(testfile, overwrite=True)
hdul.writeto(testfile_gz, overwrite=True)

In [4]:
ls -lh test.fits*

-rw-r--r-- 1 simon simon 78M May 18 15:35 test.fits
-rw-r--r-- 1 simon simon 71M May 18 15:35 test.fits.gz


In [5]:
def read(filename, decompress=False):
    if decompress:
        with gzip.open(filename, 'rb') as f:
            fd = io.BytesIO(f.read())
    else:
        fd = filename
        
    with fits.open(fd) as hdul:
        for hdu in hdul:
            # force reading data
            data = hdu.data

In [6]:
%timeit read(testfile)

12.5 ms ± 290 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
%timeit read(testfile_gz)

4 s ± 87.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%timeit gzip.open(testfile_gz).read()

352 ms ± 6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%timeit read(testfile_gz, decompress=True)

374 ms ± 6.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
